In [ ]:
#Disclaimer: This code is adapted from mscross et al. (https://github.com/mscross/pysplit)

#Initial Advice: It may be easier to generate the back trajectory files using pyPSCF instead.

"""
=====================
Trajectory Generation
=====================

PySPLIT includes a method that calls HYSPLIT to generate trajectories
launched from a single location over as many months and years as desired in
a single comprehensive batch.

=====================
Requirements
=====================

HYSPLIT: 

Unlike the majority of PySPLIT, this feature requires a local installation
of HYSPLIT, which can be acquired from the NOAA ARL
here<http://ready.arl.noaa.gov/HYSPLIT.php>_. The location of the `hyts_std` executable must be known.

Meteorology: 

The inputs are 1-degree GDAS files, retrieved either from the hysplit application or NOAA website

------------------------------------------------------------------------------------

=====================
Meteorology Input (Example)
=====================

This example uses the following GDAS files, freely available from the ARL:

If you want to generate back trajectories for this time period,
  Weeks 1 - 5 of:
  * January 2007
  * August 2007
  * January 2011
  * August 2011

The following weeks must also be included in your gdas folder (see meteo_dir):
* Weeks 4 and 5 of:
  * December 2006
  * July 2007
  * December 2010
  * July 2011

I believe this mainly pertains to if you want to generate forward trajectories
* Week 1 of:
  * February 2007
  * September 2007
  * February 2011
  * September 2011

------------------------------------------------------------------------------------

Other Information

As GDAS files are in 3 hour intervals, the very last trajectories in January
and August will not generate properly without the third group of weeks.
As a general principle, meteorology data sets corresponding to the desired time
and geospatial extent must be available to successfully generate trajectories.
These data sets must be in the ARL-packed format.  All meteorology downloaded
through the HYSPLIT ARL data FTP is already in this format, and HYSPLIT
contains some utilities to convert other file types to this format. 
Acquistion of appropriate files and the nature of
the ARL-packed format and conversion is beyond the scope of this example,
and for more information consult the official `User's
Guide<http://www.arl.noaa.gov/documents/reports/hysplit_user_guide.pdf`_
if necessary before proceeding.

Prior to attempting trajectory generation with PySPLIT, the user should ensure
that meteorology file names follow a format of '*mon*YY*#' or '*mon*YYYY*#' where:

* '*' is a Bash-style wildcard
* 'mon' is a three-letter lower-case abbreviation of the month
* 'YY' or 'YYYY' is a two or four digit integer representation of the year 
* '#' is the number of the file within the month

For example, '*jan*07*2' will match files named 'gdas1.jan2007.w2' and
'edas.jan2007_2'.

This naming convention is required because PySPLIT examines file names to
determine which meteorology files correspond to the desired date of
trajectory launch.  It is strongly recommended that users keep meteorology
files of different types/origins, like the two files in the example above,
in separate directories as PySPLIT does not differentiate between the two.

Output along-trajectory meteorology variables may be selected by interacting
with HYSPLIT.

------------------------------------------------------------------------------------

=====================
Generate Trajectories:
=====================

"""
import pysplit
"""
The following three arguments indicate locations of the HYSPLIT working directory,
the desired and existing trajectory storage directory, and the directory
that contains the meteorology files. 
"""

working_dir = r'/Users/ianmarroquin/Desktop/working'
storage_dir = r'/Users/ianmarroquin/Desktop/colgate'
meteo_dir = r'/Users/ianmarroquin/Desktop/gdas'


"""
The next argument is the basename of each trajectory i.e., trajectory filenames
consist of this basename followed by the altitude and season,
and then the year, month, day, and hour in the format YYYYMMDDHH.
"""

basename = 'colgate'
#The above name is derived from Colgate University

"""
The following arguments are the initialization information for the 
trajectory generation run:  lists or ranges of years, months, hours (in UTC,
NOT local time), and altitudes in meters above ground level.
"""

years = [2023]
months = [3]
hours = [18]
altitudes = [20]
#altitudes = [500, 1000, 1500]
location = (32.87, -117.26)
#location = (42.82, -75.54)
runtime = -48

#Campaign Dates: Feb 14, 2023 to present

pysplit.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime, 
                          monthslice=slice(0, 32, 1), get_reverse=True,
                          get_clipped=True, hysplit = "/Users/ianmarroquin/Desktop/hyts_std/hyts_std")

#Note: path of hyts_std must be known.


"""

Other Information 2

There are seven keyword arguments in this method.  The first is a slice object
that is applied to the range of days in each month, allowing us to generate
trajectories for all (default ``slice(0, 32, 1)) or a subset of days in each
month.  Here we choose to launch trajectories every other day. 

We have so far set up our run such that trajectories are launched every other
day in January and August, and their paths are calculated thence backwards
five days.  For trajectories launched at the beginning of a month, therefore,
we need meteorology data from the previous month.  The ``meteo_bookends``
indicates which meteorology files from the previous and next months are
needed.  As we are using weekly datasets, we need
weeks 4 (days 22-28) and weeks 5.  The default value (``[[4, 5], [1]]``)
also includes the first week of next month, which is critical when generating
trajectories with t=0 at the very end of the day..

The keywords ``get_reverse`` (default False) and ``get_clipped`` (default
False) control whether additional trajectory files are created.  A 'reverse'
trajectory is one launched from the endpoint of an original trajectory, and
run in the opposite direction.  A 'clipped' trajectory does not require
additional calculation, but is a copy of the original trajectory file
containing only path information.  This latter type is only strictly necessary
if using a version of HYSPLIT older than January 2017 (854), but we will acquire both
reverse and clipped trajectories anyway.

The keywords ``meteoyr_2digits`` (default True) and ``outputyr_2digits``
(default False) are critical in situations where users are dealing with
decades of meteorology files and trajectories.  In such a situation,
``meteoyr_2digits`` should be set to False.  If a user requires the old
file naming convention, ``outputyr_2digits`` should be set to True.  However,
``outputyr_2digits == False`` is preferred in general, and is required when
running non-21st century trajectories, as the file name will be the only
indication during trajectory loading of the century to which these trajectories belong.
In this example, we have only 21st century data and
our trajectory files will be generated just fine with the
default behavior of both keywords.

The final keyword argument is the location of the HYSPLIT trajectory
executable.  On Windows systems, this will usually be
'Users/ianmarroquin/Desktop' (default value).  PySPLIT calls this
executable to calculate trajectories.

Let's call our method.  Only keyword arguments with values we have decided 
to change from default are included below.  This call may take several
minutes to complete, depending on your system parameters.
"""

In [ ]:
#Troubleshooting Procedure:
#Check through all the files making sure file names are correct, no missing files or data (check size of file), or that nothing appears to be corrupted.
#You will likely need to download ftp which does no come pre-installed (to be able to download the APR files directly from hysplit app).

#If you run into the issue of missing files/files are 0 kb in size, downloading the gdas files directly from the hysplit application seemed to fix the problem.

#I had to update the directory of the of the ASCDATA.CFG file and bdyfiles folder 
#consulted the https://www.arl.noaa.gov/documents/reports/hysplit_user_guide.pdf for insight

#Note: In the example, the missing records were from the final group of weeks.


#Procedure to download brew then ftp:

#  /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"


#this correctly sets the path so you can use brew
#(echo; echo 'eval "$(/opt/homebrew/bin/brew shellenv)"') >> /Users/ianmarroquin/.zprofile
#eval "$(/opt/homebrew/bin/brew shellenv)"

# brew install inetutils
#type in ftp in the terminal to make sure it is properly installed.


#use help(pysplit.generate_bulktraj) for additional insights
